# Proyecto Final Señales y Sistemas 2025 -2

## **Objetivo**: Implementar técnicas de representación en tiempo y frecuencia para el reconocimiento de señales de electroencefalografía (EEG) en tareas de imaginación motora (Motor Imagery)


![eegMI](https://figures.semanticscholar.org/288a54f091264377eccc99a19079c9387d66a78f/3-Figure2-1.png)

Las señales de EEG pueden ser ruidosas debido a diversas fuentes, incluidos artefactos fisiológicos e interferencias electromagnéticas. También pueden variar de persona a persona, lo que dificulta la extracción de características y la comprensión de las señales. Además, esta variabilidad, influenciada por factores genéticos y cognitivos, presenta desafíos para el desarrollo de soluciones independientes del sujeto. 

**Base de datos**: GiGaScience Database [https://gigadb.org/dataset/100295](https://gigadb.org/dataset/100295)

Ver Sección 3.1 en [Multimodal Explainability Using Class Activation Maps and Canonical Correlation for MI-EEG Deep Learning Classification](https://www.mdpi.com/2076-3417/14/23/11208)


## Instalamos las librerias necesarias

## Ejercicio 1
Consultar para qué sirven las siguientes librerías

In [ ]:
#!pip install tensorflow==2.15.0
!pip install mne==1.6.0
!pip install braindecode===0.7
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases

### Tensorflow:
Para saber que es tensorflow primero hay que conocer que es un tensor, y la mejor forma de ilustrarlo es por ejemplos: si un escalar (un número solo) es un tensor de orden 0, un vector (dupla, o n-upla) es un tensor de orden 1, una matriz es un tensor de orden 2. Podríamos decir entonces que un tensor es en general un arreglo de números, que puede ser de dimensión n. Así como la recta numérica es la versión de una dimensión del plano cartesiano 2D que a su vez también puede haber plano cartesiano de 3 dimensiones.
Ahora sabiendo que es un tensor, tensorflow es una librería que permite hacer cálculos con tensores, esto es lo que ha permitido y ha impulsado la construcción entrenamiento y despliegue de Redes neuronales artificiales, Transformers y en general las nuevas tecnologías con IA.
### MNE:
MNE significa “Magnetoencephalography and Electroencephalography”. Es una librería especializada en el análisis de señales cerebrales como el EEG, es un ecosistema de herramientas para neurociencia, MNE ayuda con la carga y procesamiento de la EEG, su filtrado digital, la extracción de características (como saber que ritmo cerebral tiene una señal), visualización como topografías o mapas de calor.
### Braindecode: 
Librería en Python para clasificación, regresión y segmentación de señales EEG usando redes neuronales. Braindecode permite el entrenamiento de modelos de lenguaje, algo como ChatGPT, Braindecode convierte la EEG en tensores listos para entrenar una IA que detecte patrones que no hemos visto antes
### Python-gcpds.databases:
Es un paquete de Python que contiene Data loaders, interfaces y acceso fácil a bases de datos EEG usadas por GCPDS, pensado para proyectos de BCI, clasificación de EEG e investigación sobre señales cerebrales.


## Importamos algunas librerias necesarias

In [ ]:
from scipy.signal import resample
from scipy.signal import freqz, filtfilt, resample
from scipy.signal import butter as bw
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from gcpds.databases import GIGA_MI_ME
from sklearn.base import BaseEstimator, TransformerMixin

**Librerías de Procesamiento de Señales (SciPy.signal)**

```SciPy``` es la librería fundamental de Python para el cálculo científico. El módulo ```scipy.signal``` contiene herramientas esenciales para el procesamiento digital de señales (DSP).

**```scipy.signal.resample```**

*¿Para qué sirve?*

Remuestreo (Resampling): Cambia la frecuencia de muestreo de una señal.Es útil para downsampling (reducir la tasa de muestreo, lo que disminuye el tamaño del archivo) o upsampling (aumentar la tasa de muestreo) para sincronizar señales grabadas con diferentes equipos.

*Uso Típico:*

Ajustar los datos EEG de diferentes bases de datos a una frecuencia común (ej., de $1000$ Hz a $250$ Hz) antes de alimentar un modelo de machine learning.

**```scipy.signal.freqz```**

*¿Para qué sirve?*

Respuesta en Frecuencia de un Filtro: Calcula la respuesta en magnitud y fase de un filtro digital (definido por sus coeficientes $b$ y $a$).Permite visualizar y verificar el desempeño de un filtro (por ejemplo, butter que se explica a continuación) antes de aplicarlo a los datos.

*Uso Típico:*

Graficar el espectro de atenuación de un filtro pasabanda para asegurar que solo las frecuencias deseadas pasen a través de él.

**```scipy.signal.filtfilt```**

*¿Para qué sirve?*

Filtrado de Orden Cero (Zero-Phase Filtering): Aplica un filtro digital a los datos dos veces: una vez hacia adelante y una vez hacia atrás.Esto es crucial porque elimina el desplazamiento de fase (retraso) que introduce el filtrado tradicional unidireccional, lo que es vital en el EEG para mantener la alineación temporal de los eventos cerebrales (ERP).

*Uso Típico:*

Aplicar un filtro paso alto para eliminar la deriva basal o un filtro de banda eliminada (notch) para suprimir el ruido de la línea eléctrica ($50/60$ Hz) sin distorsionar la forma de onda.

**```scipy.signal.butter as bw```**

*¿Para qué sirve?*

Diseño de Filtros Butterworth: Genera los coeficientes (b y a) necesarios para implementar un filtro Butterworth.Los filtros Butterworth son conocidos por tener la respuesta más plana en la banda de paso (la región de frecuencias que dejan pasar), minimizando la distorsión de la señal útil.

*Uso Típico:*

Crear los coeficientes para un filtro que se aplicará posteriormente con filtfilt. Por ejemplo, para crear un filtro pasabanda entre $0.5$ Hz y $40$ Hz para el análisis EEG.


**Librerías de Ciencia de Datos y Auxiliares**

```pandas (pd)```, ```numpy (np)```, ```random```

**pandas**: La librería esencial para la manipulación y análisis de datos en formato tabular (*DataFrames*). Se usa para gestionar metadatos o la información de los sujetos.

**numpy**: El corazón de la computación numérica en Python. Se usa para manejar *arrays multidimensionales* (matrices), que es la estructura de datos que representa las señales (ej. Canales $\times$ Puntos de Tiempo).

**random**: Módulo para generar *números aleatorios*. Útil para dividir datos aleatoriamente en conjuntos de entrenamiento y prueba o para inicializar pesos de modelos.

```matplotlib.pyplot as plt```

*¿Para qué sirve?*

Visualización y Gráficos: Es la librería estándar para la creación de gráficos estáticos, interactivos y animaciones en Python.

*Uso Típico*:

Graficar las señales EEG en el dominio del tiempo o del dominio de la frecuencia (ej. la respuesta de freqz).

**Librerías Específicas del Dominio**

```gcpds.databases.GIGA_MI_ME```

¿Para qué sirve?

Esta es la librería del UN-GCPDS que mencionamos anteriormente. Específicamente, parece estar importando una clase o función que facilita el acceso y la descarga de una base de datos específica de EEG: GIGA-MI-ME (probablemente GIGA-Motor Imagery/Motor Execution).

Uso Típico:

Descargar o acceder de forma estandarizada a los datos brutos de EEG para el entrenamiento de interfaces cerebro-computadora (BCI) basadas en imaginación motora.

8. sklearn.base.BaseEstimator, TransformerMixin
¿Para qué sirve?

Estructuración de Código (Scikit-learn): Estos son las clases base que se utilizan en la librería scikit-learn para crear objetos personalizados que se comportan como otros componentes de machine learning (ej. clasificadores, pipelines).

Uso Típico:

Permite a los desarrolladores crear sus propias clases de preprocesamiento de datos (ej. una clase llamada MNEToNumpyTransformer que encapsule todo el preprocesamiento MNE) que pueden ser integradas en un pipeline de scikit-learn junto con un clasificador.

## Funciones necesarias para el preprocesamiento leve de los datos

In [ ]:
def load_GIGA(db,
              sbj,
              eeg_ch_names,
              new_fs,
              fs,
              f_bank=None,
              vwt=None,           
              run=None):
#Cargas el EEG, escoges ciertos canales,
#tomas solo left/right MI, reorganizas y si quieres cambias la frecuencia de muestreo.
    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    #tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    if run == None:
        X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    else:
        X, y = db.get_run(run, classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:, index_eeg_chs, :] #spatial rearrangement
    #X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        pass#print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)

    return X, y

def butterworth_digital_filter(X, N, Wn, btype, fs, axis=-1, padtype=None, padlen=0, method='pad', irlen=None):
#Aplica un filtro Butterworth pasa-bajas/altas/banda a una señal EEG sin producir desfase.
  """
  Apply digital butterworth filter
  INPUT
  ------
  1. X: (D array)
    array with signals.
  2. N: (int+)
    The order of the filter.
  3. Wn: (float+ or 1D array)
    The critical frequency or frequencies. For lowpass and highpass filters, Wn is a scalar; for bandpass and bandstop filters, Wn is a length-2 vector.
    For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”).
    If fs is not specified, Wn units are normalized from 0 to 1, where 1 is the Nyquist frequency (Wn is thus in half cycles / sample and defined as 2*critical frequencies / fs). If fs is specified, Wn is in the same units as fs.
  4. btype: (str) {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}
    The type of filter
  5. fs: (float+)
    The sampling frequency of the digital system.
  6. axis: (int), Default=1.
    The axis of x to which the filter is applied.
  7. padtype: (str) or None, {'odd', 'even', 'constant'}
    This determines the type of extension to use for the padded signal to which the filter is applied. If padtype is None, no padding is used. The default is ‘odd’.
  8. padlen: (int+) or None, Default=0
    The number of elements by which to extend x at both ends of axis before applying the filter. This value must be less than x.shape[axis] - 1. padlen=0 implies no padding.
  9. method: (str), {'pad', 'gust'}
    Determines the method for handling the edges of the signal, either “pad” or “gust”. When method is “pad”, the signal is padded; the type of padding is determined by padtype
    and padlen, and irlen is ignored. When method is “gust”, Gustafsson’s method is used, and padtype and padlen are ignored.
  10. irlen: (int) or None, Default=nONE
    When method is “gust”, irlen specifies the length of the impulse response of the filter. If irlen is None, no part of the impulse response is ignored.
    For a long signal, specifying irlen can significantly improve the performance of the filter.
  OUTPUT
  ------
  X_fil: (D array)
    array with filtered signals.
  """
  b, a = bw(N, Wn, btype, analog=False, output='ba', fs=fs)
  return filtfilt(b, a, X, axis=axis, padtype=padtype, padlen=padlen, method=method, irlen=irlen)

class TimeFrequencyRpr(BaseEstimator, TransformerMixin):
#toma tu EEG crudo

#lo divide en bandas de frecuencia (con Butterworth)

#lo corta en ventanas temporales

#lo devuelve en forma de tensor 4D o 5D

#listo para meter a una red neuronal
  """
  Time frequency representation of EEG signals.

  Parameters
  ----------
    1. sfreq:  (float) Sampling frequency in Hz.
    2. f_bank: (2D array) Filter banks Frequencies. Default=None
    3. vwt:    (2D array) Interest time windows. Default=None
  Methods
  -------
    1. fit(X, y=None)
    2. transform(X, y=None)
  """
  def __init__(self, sfreq, f_bank=None, vwt=None):
    self.sfreq = sfreq
    self.f_bank = f_bank
    self.vwt = vwt
# ------------------------------------------------------------------------------

  def _validation_param(self):
    """
    Validate Time-Frequency characterization parameters.
    INPUT
    -----
      1. self
    ------
      2. None
    """
    if self.sfreq <= 0:
      raise ValueError('Non negative sampling frequency is accepted')


    if self.f_bank is None:
      self.flag_f_bank = False
    elif self.f_bank.ndim != 2:
      raise ValueError('Band frequencies have to be a 2D array')
    else:
      self.flag_f_bank = True

    if self.vwt is None:
      self.flag_vwt = False
    elif self.vwt.ndim != 2:
      raise ValueError('Time windows have to be a 2D array')
    else:
      self.flag_vwt = True

# ------------------------------------------------------------------------------
  def _filter_bank(self, X):
    """
    Filter bank Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_f: (4D array) set of filtered EEG signals, shape (trials, channels, time_samples, frequency_bands)
    """
    X_f = np.zeros((X.shape[0], X.shape[1], X.shape[2], self.f_bank.shape[0])) #epochs, Ch, Time, bands
    for f in np.arange(self.f_bank.shape[0]):
      X_f[:,:,:,f] = butterworth_digital_filter(X, N=5, Wn=self.f_bank[f], btype='bandpass', fs=self.sfreq)
    return X_f

# ------------------------------------------------------------------------------
  def _sliding_windows(self, X):
    """
    Sliding Windows Characterization.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
    OUTPUT
    ------
      1. X_w: (4D array) shape (trials, channels, window_time_samples, number_of_windows)
    """
    window_lenght = int(self.sfreq*self.vwt[0,1] - self.sfreq*self.vwt[0,0])
    X_w = np.zeros((X.shape[0], X.shape[1], window_lenght, self.vwt.shape[0]))
    for w in np.arange(self.vwt.shape[0]):
        X_w[:,:,:,w] = X[:,:,int(self.sfreq*self.vwt[w,0]):int(self.sfreq*self.vwt[w,1])]
    return X_w

# ------------------------------------------------------------------------------
  def fit(self, X, y=None):
    """
    fit.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, time_samples)
      2. y: (1D array) target labels. Default=None
    OUTPUT
    ------
      1. None
    """
    pass

# ------------------------------------------------------------------------------
  def transform(self, X, y=None):
    """
    Time frequency representation of EEG signals.
    INPUT
    -----
      1. X: (3D array) set of EEG signals, shape (trials, channels, times)
    OUTPUT
    ------
      1. X_wf: (5D array) Time-frequency representation of EEG signals, shape (trials, channels, window_time_samples, number_of_windows, frequency_bands)
    """
    self._validation_param()     #Validate sfreq, f_freq, vwt

    #Avoid edge effects of digital filter, 1st:fbk, 2th:vwt
    if self.flag_f_bank:
        X_f = self._filter_bank(X)
    else:
        X_f = X[:,:,:,np.newaxis]

    if self.flag_vwt:
      X_wf = []
      for f in range(X_f.shape[3]):
        X_wf.append(self._sliding_windows(X_f[:,:,:,f]))
      X_wf = np.stack(X_wf, axis=-1)
    else:
      X_wf = X_f[:,:,:,np.newaxis,:]

    return X_wf

#plot eeg   
def plot_eeg(X,tv,ax,channels,esp=2,title=None):
    # X in CH x Samples
    n_canales = X.shape[0]

    for ch in range(n_canales): # canales
            xx = X[ch]
            xx = xx - np.mean(xx)
            xx = xx/np.max(abs(xx))
            ax.plot(tv, xx +(ch * esp), label=channels[ch])  # Desplazamos cada canal para visualización
    ax.set_yticks(range(0, esp * n_canales, esp), channels)  # Etiquetas en el eje Y
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Canales EEG [$\mu$V]")
    ax.set_title(title)
    ax.grid(True)
    ax.set_xlim([min(tv)-0.01,max(tv)+0.01])
    ax.set_ylim([-esp,n_canales*esp+0.01])





      

## Establecemos el protocolo de pruebas y la configuración del montaje EEG

Describir el protocolo de captura de datos y el montaje utilizado


![mi](https://www.mdpi.com/diagnostics/diagnostics-13-01122/article_deploy/html/images/diagnostics-13-01122-g001.png)
![montaje](https://www.mdpi.com/applsci/applsci-14-11208/article_deploy/html/images/applsci-14-11208-g001.png)

In [ ]:
channels = ['Fp1','Fpz','Fp2',
            'AF7','AF3','AFz','AF4','AF8',
            'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
            'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
            'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
            'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
            'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
            'PO7','PO3','POz','PO4','PO8',
            'O1','Oz','O2',
            'Iz']

areas = {
    'Frontal': ['Fpz', 'AFz', 'Fz', 'FCz'],
    'Frontal Right': ['Fp2','AF4','AF8','F2','F4','F6','F8',],
    'Central Right': ['FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8',],
    'Posterior Right': ['P2','P4','P6','P8','P10','PO4','PO8','O2',],
    #'Central': ['Cz'],
    'Posterior': ['CPz','Pz', 'Cz','POz','Oz','Iz',],
    'Posterior Left': ['P1','P3','P5','P7','P9','PO3','PO7','O1',],
    'Central Left': ['FC1','FC3','FC5','FT7','C1','C3','C5','T7','CP1','CP3','CP5','TP7',],
    'Frontal Left': ['Fp1','AF3','AF7','F1','F3','F5','F7',],
}

arcs = [
    #'hemispheres',
    'areas',
    'channels',
]

## Definimos la ruta y los argumentos para la carga de los datos de EEG

In [ ]:
db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
#ti = 0
#tf = 7
new_fs = 256.
load_args = dict(db = db,
                 eeg_ch_names = channels,
                 fs = db.metadata['sampling_rate'],
                 #f_bank = np.asarray([[4., 40.]]),
                 #vwt = np.asarray([[ti, tf]]), #2.5 - 5 MI
                 new_fs = new_fs)

## Cargamos los datos según el sujeto que se quiera

Si se quiere cargar los datos de todos los sujetos, aplicar un ciclo que itere la lista de sujetos y de esta forma se cargara uno por uno dependiendo lo que se desee realizar.

Por ejemplo:

for i in sbj:
    X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
sbj = 5
X, y = load_GIGA(sbj=sbj, **load_args)

In [ ]:
print(f'X con {X.shape[0]} intentos; {X.shape[1]} canales; {X.shape[2]} muestras No. de segundos {X.shape[2]/new_fs}')

In [ ]:
X.shape

## Visualización de las señales de EEG en el tiempo

In [ ]:
#graficar canales promedio
trial = 0
ti = 0 # ti
tf = 7 # tf
tv = np.arange(ti,tf,1/new_fs)

#Señal cruda
fig,ax = plt.subplots(1,1,figsize=(8,8),sharex = True)
# Graficar cada canal en un subplot banda respectiva

plot_eeg(X[trial],tv,ax=ax,channels=channels,title='EEG original')
plt.show()

# Ejercicio 2

Discuta la gráfica anterior

### R=/

Nuestro compañero Diego nos explico que hay una teoria que dice que los reflejos, o las señales se originan desde el tronco encefalico, o desde la parte trasera del cerebro, y a partir de esto los impulsos se transmiten hacia adelante.

Notando que la organización de las señales detectadas por los electrodos esta justamente siguiendo el patrón antes descrito pudimos visualizar que la gráfica tiene un comportamiento similar a lo descrito por esa teoria, donde los cambios en las señales cerebrales suceden primero en los electrodos de más atras de la cabeza y a partir de ahi hay un leve retardo en las señales de los de mas adelante.

También es posible notar que el estimulo se ve reflejado en las señales a partir aproximadamente antes del segundo 3.

Parece que el estimulo se mantiene luego de comenzar ya que luego de dicho cambio la señales no vuelven precisamente a su estado original sino que se mantienen con dicho comportamiento.

Nota: Discuta en qué consisten los ritmos cerebrales

![montaje](https://cdn.shopify.com/s/files/1/0348/7053/files/storage.googleapis.com-486681944373284_61cb9936-f6c2-493d-8402-3426d7f5a049_1024x1024.jpg?v=1689309340)



## ¿En que consisten los ritmos cerebrales?

Las neuronas en nuestro cerebro funcionan por medio de pequeñas corrientes electricas, según se ha estudiado dichas corrientes son variantes en el tiempo y tienen oscilasciones como las de una onda sinusoidal sin ser exactamente sinusoidales. Al parecer, la frecuencia de nuestras señales electricas cambia según en que estado mental nos encontramos, a esto es alo que se llama ritmo cerebral, y según estudios se clasifican como:

**Delta:** Sueño profundo (0,5-4 Hz)

**Theta:** creatividad, sueño ligero (4-8 Hz)

**Alpha:** relajación (8-12 Hz)

**Beta:** concentración (13-30 Hz)

**Gamma:** Atención sostenida, integración sensorial, aprendizaje (30-100 Hz)

**High Gamma:** Procesamiento sensorial detallado, actividad durante tareas cognitivas exigentes (80-150 Hz aprox)

**HFO:** Ripples (consolidación de memoria en hipocampo, actividad cercana a la epilepsia 80-250 Hz), Fast Ripples (actividad que genera las crisis de epilepsia 250-500 Hz) y hasta de más de 500 Hz pero solo son visibles en investigación invasiva

De estas. las últimas 2 no nos interesan para este proyecto, ya que no se suelen estudiar en EEG convencional al ser dificiles de captar con EEG de superficie.



In [ ]:
# filtramos trials completos en ritmos cerebrales utilizando filtros IIR


f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]]) # definición del banco de filtros, define las cinco bandas de frecuencia que 
#se suelen estudiar en EEG
vwt = np.asarray([[ti, tf]]) #2.5 - 5 MI 0 - 7 trial completo #define el segmento temporal del trial que se quiere extraer
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank) #crea un filtro pasa banda por cada banda de EEG, devuelve en forma de tensores 4D o 5D

Xrc = np.squeeze(tf_repr.transform(X)) # toma la señal original, se filtra en cada banda definida, se obtiene una representación por banda 

Xrc.shape

# Ejercicio 3

Expliqué cómo se calcularon cada una de las 5 dimensiones del arreglo Xrc

### R=/
Transform es un método al cual se ingresa X el cual tiene la forma:

X.shape = (trials, channels, times)

Y fue traido desde la base de datos de EEG con load_GIGA

Ahora el método transform filtra por el banco de bandas que ya establecimos antes dando como resultado X_f que tiene la forma:

X_f.shape = (T, C, S, B) donde B es el número de bandas

Se supone que a continuación añade una dimension adicional que corresponde a las ventanas temporales, eso da como resultado:

X_wf.shape = (T, C, S, 1, B) en este caso es 1 por lo que no se usa vwt

Asi finalmente se obtiene que Xrc es:

Xrc.shape = (trials, channels, window_time_samples, number_of_windows, frequency_bands)



In [ ]:
import matplotlib.pyplot as plt

ritmo = ['delta','theta','alpha','beta','gamma']
trial = 0
n_trials, n_canales, n_muestras, n_bands = Xrc.shape  # Simulación de datos

esp = 2 #espaciado canales
fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    plot_eeg(Xrc[trial,:,:,b],tv,ax=ax[b],channels=channels,title=f'EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
plt.show()

## Visualización de las señales de EEG en la frecuencia

In [ ]:
#señal orignal
Xwo = np.fft.rfft(X,axis=-1)
vfreq = np.fft.rfftfreq(X.shape[2],1/new_fs)

Xwo.shape
plt.plot(vfreq,20*np.log10(np.abs(Xwo[trial])).T)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud [dB]')
plt.title('Eespectro Señal EEG original')
plt.show()


## Ejercicio 4

Discuta la gráfica anterior

### R=/

La gráfica muestra el espectro de las señales de EEG que se están estudiando en esta ocasión. Algo que se puede notar a simple vista es un pico en la frecuencia de 60 Hz. Es posible que dicho pico sea ruido producido por la red eléctrica. Esto coincide con lo que se investigo sobre las EEG donde se habla de que es posible ver este ruido en las EEG y por ese motivo se suele aplicar un filtrado a dicha frecuencia.

También es posible notar un gran pico apenas al inicio del espectro, esto coincide con las señales que se ven en las ondas filtradas, donde en frecuencias bajas muy al inicio de la prueba hay una oscilación más pronunciada.

En general el espectro tiene  una magnitud mayor en frecuencias bajas, nuevamente esto es algo que coincide con las gráficas de más arriba que muestran ondas más pronunciadas en el EEG filtrado hasta antes de lo 13 Hz

In [ ]:
#espectro señales filtradas
Xwb = np.fft.rfft(Xrc,axis=2)

Xwb.shape

In [ ]:
#espectro señales filtradas por bandas - ritmos cerebrales

fig,ax = plt.subplots(5,1,figsize=(8,40))
# Graficar cada canal en un subplot banda respectiva
for b in range(f_bank.shape[0]): #bandas
    ax[b].plot(vfreq,20*np.log10(np.abs(Xwb[trial,:,:,b])).T)
    ax[b].set_xlabel('Frecuencia [Hz]')
    ax[b].set_ylabel('Magnitud [dB]')
    ax[b].set_title(f'Esepctro EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]}')
    
plt.show()

## Ejercicio 5

Discuta las gráficas

### R=/

Las gráficas muestran el comportamiento del filtro en cada una de las bandas seleccionadas. La primera gráfica muestra un filtrado que pasa frecuencias bajas sin apagar por completo lás más altas, esto se visualiza en el pronunciado pico muy al inicio del espectro para su posterior suavidad.

Lo mismo pasa en las siguientes gráficas, lo que visualizamos siempre es el comportamiento del filtro pasa bandas, en cada una de las bandas seleccionadas.

## Visualización de espectrogramas

Consultar qué es la Short Time Fourier Transform

### R=/

La Short-Time Fourier Transform (STFT), o Transformada de Fourier de Tiempo Corto, es un método para analizar cómo cambia el contenido en frecuencia de una señal a lo largo del tiempo.

En términos simples:

Tomas la señal continua (por ejemplo, un EEG).

La divides en ventanas pequeñas (segmentos cortos en el tiempo).

En cada ventana aplicas la Transformada de Fourier.

Obtienes un espectro de frecuencias para cada instante temporal.

El resultado es una representación tiempo–frecuencia.

La transformada de Fourier tradicional te dice qué frecuencias hay, pero no cuándo ocurren.

La STFT te da ambas cosas:

Tiempo

Frecuencia

**Como funciona:**

Seleccionas una ventana (por ejemplo, 256 muestras).

Tomas ese segmento y aplicas la FFT.

Mueves la ventana unos pasos (overlap).

Repites la FFT en cada ventana.

Cada ventana produce un espectro de frecuencias:

El conjunto de todos los espectros forma un espectrograma.



In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
from scipy.signal import stft #
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('C4')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Esepctrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')


# Ejercicio 6

Presente las gráficas de stft para distintos canales en los 5 ritmos cerebrales y discuta.

In [ ]:
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('Iz')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 0
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 1
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 3
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 4
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

### Interpretación:

Las gráficas que estamos viendo en espectro tienen una forma interesante de expresar 3 cosas:

Primero en el eje X vemos el tiempo

Segundo en el eje Y vemos la frecuencia

Y tercero según el color, tal como se ve en la escala de gradiente de color, en cuanto más amarilo significa que hay una mayor magnitud de la frecuencia 

Eso quiere decir que por ejemplo en la gráfica de el EEG del canal Iz filtrado a ritmo Gamma (32-100 Hz) la magnitud de las frecuencias es mayor entre 32 y un poco más de los 100 Hz (se ve en la colorimetria, ese color amarillo y verde quiere decir más magnitud) Y ademas hay una linea amarillo profundo más arriba del 50, pero debajo del 100, probablemente 60 Hz, tal como habiamos dicho antes quizás producto del ruido de la red eléctrica.

Eso mismo sucede en las demás gráficas, nuevamente se ve el comportamiento del filtro, solo que esta vez nos permite percatarnos de en que momento del tiempo hay una cierta frecuencia de manera más o menos intensa en magnitud.

Otro comentario que podemos hacer es respecto al espectrograma de toda la señal (la señal original) donde se ve una mayor magnitud en las frecuencia bajas algo visible por el profundo color amarillo abajo en la gráfica y los colores más oscuros y cercanos al azul en las gráficas de más arriba.

Ahora bien, si vemos las gráficas de más arriba que nos permiten visualizar las oscilaciones en distintos canales podemos ver que Iz tiene oscilaciones al principio del intervalo. Para visualizar una señal que tenga oscilaciones distintas y en otro momento del intervalo ahora vamos a visualizar el espectrograma del canal Fp1

In [ ]:
nperseg = 0.5*new_fs#longitud ventas en muestras
vfs,t,Xstft = stft(X,fs=new_fs,nperseg=nperseg,axis=2)
Xstft = 20*np.log10(abs(Xstft))

#graficar stft para un trial y un canal
trail = 0
chi = channels.index('Fp1')

fig, ax = plt.subplots(2, 1,figsize=(10,6))

ax[1].plot(tv,X[trail,chi,:])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstft[trail,chi])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Original -- Ch = {channels[chi]}')
print(Xstft.shape)

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 0
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 1
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 2
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 3
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

In [ ]:
#estimar stft con ventanas de nperseg puntos sobre eje temporal en EEG original
b = 4
vfs,t,Xstftb = stft(Xrc,fs=new_fs,nperseg=nperseg,axis=2)
Xstftb = 20*np.log10(abs(Xstftb))

print(Xstftb.shape)


fig, ax = plt.subplots(2, 1,figsize=(10,6))
ax[1].plot(tv,Xrc[trail,chi,:,b])
ax[1].set_ylabel("Amp. [$\mu$ V]")
im = ax[0].pcolormesh(t, vfs, Xstftb[trail,chi,:,b,:])
fig.colorbar(im, ax=ax[0],orientation="horizontal",pad=0.2)
plt.gca()
plt.xlabel('t [seg]')
plt.ylabel('f [Hz]')
ax[0].set_title(f'Espectrograma EEG Filtrado {f_bank[b,0]}-{f_bank[b,1]} [Hz] -- Ritmo: {ritmo[b]} -- Ch = {channels[chi]}')

### Interpretación de este canal:

Algo que nos permite ver esta nueva serie de gráficas es que el método que esta usando para el filtrado funciona, y que no estamos mostrando sencillamente gráficas genericas, ya que varia bastante con relación al canal Iz. Al igual que el canal Iz este espectrograma muestra una linea en los 60 Hz debido al ruido de la red electrica y muestra que la mayoria de su espectro esta en frecuencias bajas. Probablemente el sujeto de pruebas se encontraba en relajación a la hora de realizar la toma de muestras y hubieron cambios en sus ritmos cerebrales en  4 momentos en especifico: uno entre 0 y 1 segundos, otro entre 1 y 2 segundos, el tercero entre 2 y 3 segundos y el último hacia el final de la prueba entre 6 y 7 segundos. Por los espectrogramas es posible ver que el cambio de un ritmo cerebral a otro no es instantaneo sino gradual, pues alrededor de donde hay grandes oscilasciones en frecuencias altas es posible ver en los espectrogramas de más arriba oscilaciones en la otras frecuencias pronunciadas.

## Visualización de señales EEG sobre montaje 10-20

In [ ]:
import mne

# Cargar el montaje estándar
easycap_montage = mne.channels.make_standard_montage("standard_1020")


# Crear un montaje personalizado con los electrodos seleccionados
custom_pos = {ch: easycap_montage.get_positions()["ch_pos"][ch] for ch in channels}
custom_montage = mne.channels.make_dig_montage(ch_pos=custom_pos, coord_frame="head")

# Mostrar el montaje personalizado
custom_montage.plot(show_names=True)
fig = custom_montage.plot(kind="3d", show_names=True, show=False)
fig.gca().view_init(azim=70, elev=15)  # Ajustar la vista 3D

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.visualizations.git

# Topomaps

In [ ]:
from gcpds.visualizations.topoplots import topoplot


trial = 150
vec_topo_o = abs(X[trial,:]).mean(axis=-1)
vec_topo_b = abs(Xrc[trial,:,:,:]).mean(axis=1)


fig,ax = plt.subplots(1,6,figsize=(20,10))
topoplot(vec_topo_o, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[0],show=False,vlim=(min(vec_topo_o), max(vec_topo_o)))

for b in range(f_bank.shape[0]):
    vec_ = vec_topo_b[:,b]
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[b+1],show=False,vlim=(min(vec_), max(vec_)))
    ax[b+1].set_title(ritmo[b])    

ax[0].set_title(f'EEG-suj={sbj}-trial={trial}')    

plt.show()

## Ejercicio 7

Discuta

### R=/

Para poder discutir sobre las anteriores gráficas es necesario saber de que se tratan, **¿Que son los Topomaps?**

Un topomap en EEG (topographic map o mapa topográfico) es una representación espacial del voltaje o potencia eléctrica en el cuero cabelludo, mostrada como un mapa de calor sobre una vista 2D de la cabeza.

En términos sencillos:
Un topomap te muestra “dónde” en el cerebro o cuero cabelludo está ocurriendo la actividad eléctrica, no solo “cuándo” o “qué frecuencia”.

Ahora sabiendo esto, **Discutamos sobre estas graficas:**

En la primera gráfica se muestra que la mayoria de la actividad se realiza en puntos especificos de la cabeza: entre los sensores F1 y Fz, en el sensor C6T8, en el sensor P10, en AF7, entre C1 y CP1, en C5 y en F6

En la segunda se ve que la mayoria de actividad delta sucede en C6T8 y en O1.

La tercera muestra que la gran m,ayoria de actividad Theta sucede al frente de la cabeza, al igual que la actividad Alpha lo que se ve en la cuarta gráfica.

La quinta muestra que la mayoria de la cabeza tiene buena actividad Beta, pero nuevamente hay un punto de concentración, los sensores C6T8.

Por último la sexta muestra que casi toda la actividad Gamma sucede en C6T8}

A lo largo de las gráficas se nota que entre P1 y POz hay un sitio de muy baja actividad cerebral.

Las gráficas también nos permiten concluir que por algún motivo u otro, para lo que sea que vaya a realizarse con estas EEG resulta muy importante los puntos entre C6T8, ya que es un área de alta actividad.

Teniendo en cuenta esto, creemos que lo que se hizo en esta prueba fue probablemente intentar mover la mano o pierna izquierda del sujeto de pruebas ya que según lo investigado la actividad cerebral es mayor en el lado contrario a la extremidad que se quiere mover.

## Common Spatial Patterns

Consulté qué son los Common Spatial Patterns (CSP) y su aplicación al procesado de señales EEG

### R=/

**¿Que son los Common Spatial Patterns? (CSP)**

Normalmente en EEG se tienen muchos canales(electrodos), señales ruidosas y diferencias entre estimulos también llamdos clases muy sutiles, por ejemplo, la diferencia entre las señales de mover la mano derecha y mover la mano izquierda suelen tener diferencias dificiles de ver.

Lo que hace el CSP es buscar una cierta combinación de electrodos que separe bien las clases o estimulos.

¿Como separa bien las clases o estimulos? Busca cierta combinación de electrodos donde la varianza (potencia) en una sea grande en un clase y pequeña en la otra.

Es como si el cerebro tocara una orquesta para expresar derecha y una orquesta para expresar izquierda y el CSP se da cuenta de que en la orquesta de izquierda el violin toca mucho más, y es más dominante, mientras que en la orquesta de la derecha el violin casi no varia su melodia y solo hace voces.

**Aplicación al procesado de señales EEG**

Su aplicación número uno es: Extraer características discriminantes para clasificar estados cerebrales a partir de EEG multicanal.
Es decir que:
Tienes muchas señales mezcladas

CSP encuentra patrones espaciales

Esos patrones convierten el EEG en números separables

Esto se usa en interfaces cerebro computador (BCI) y en clasificación de estados mentales.

In [ ]:
import mne
from mne.decoding import CSP

# Instancia del objeto CSP
n_components = 2
csp = CSP(n_components=n_components, log= True, transform_into='average_power')
# Ajuste y transformación de los datos
csp_data = csp.fit_transform(X.astype(np.float64), y)

In [ ]:
print("CSP Transformado Shape:", csp_data.shape)
plt.scatter(csp_data[:,0],csp_data[:,1],c=y)
plt.show()

In [ ]:
#EEG original
fig,ax = plt.subplots(1,n_components,figsize=(5,5))
for cc in range(n_components):
    vec_ = np.abs(csp.filters_[cc])
    topoplot(vec_, channels, contours=3, cmap='Reds', names=channels, sensors=False,ax=ax[cc],show=False,vlim=(min(vec_), max(vec_)))
    ax[cc].set_title(f'CSP {cc+1}') 


In [ ]:
#lectura de datos
sbj = 14
X, y = load_GIGA(sbj=sbj, **load_args)

f_bank = np.array([[0.5,4.],[4., 8.],[8.,13.],[13.,32.],[32.,100.]])
vwt = np.array([[0.25, 1.75],[1.5,3],[2.75,4.25],[4,5.5],[5.25,6.75]]) #2.5 - 5 MI 0 - 7 trial completo
tf_repr = TimeFrequencyRpr(sfreq = new_fs, f_bank = f_bank,vwt=vwt)
X_ = np.squeeze(tf_repr.transform(X))
X_.shape

In [ ]:
# csp por ventanas y ritmos
# Definir las dimensiones del arreglo
ritmos_ = f_bank.shape[0] 
ventanas_ = vwt.shape[0]
n_comp = 2
# Inicializar el arreglo vacío con listas anidadas
csp_M = [[None for _ in range(ventanas_)] for _ in range(ritmos_)]
csp_filters_ = np.zeros((ritmos_,ventanas_,X_.shape[1],X_.shape[1])) #ritmos ventanas Ch
Xcsp_ = np.zeros((X_.shape[0],n_comp,ritmos_,ventanas_))

for i in range(ritmos_):
    for j in range(ventanas_):
        print(f'CSP ritmo {f_bank[i]} -- ventana {vwt[j]}...')
        csp_M[i][j] =  CSP(n_components=n_comp, log= True, transform_into='average_power')
        Xcsp_[:,:,i,j] = csp.fit_transform(X_[:,:,:,j,i].astype(np.float64), y)
        csp_filters_[i,j,:] = np.abs(csp.filters_) 

In [ ]:
# graficar topomaps
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        vec_ = csp_filters_[i,j,0]
        vec_ = vec_/max(vec_)
        topoplot(vec_, channels, contours=3, cmap='Reds', names=None, sensors=False,ax=ax[i,j],show=False,vlim=(min(vec_), max(vec_)))
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)   
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)
    
plt.subplots_adjust(hspace=-0.025,wspace=-0.025)    
plt.show()      

In [ ]:
#scatters
fig, ax = plt.subplots(ritmos_,ventanas_,figsize=(12,12))

for i in range(ritmos_):
    for j in range(ventanas_):
        ax[i,j].scatter(Xcsp_[:,0,i,j],Xcsp_[:,1,i,j],c=y)
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
    ax[i,0].set_ylabel(ritmo[i],fontsize=20)   
for j in range(ventanas_):
     ax[0,j].set_title(f'{vwt[j,0]}--{vwt[j,1]} [s]',fontsize=15)
    
plt.subplots_adjust(hspace=0.1,wspace=0.1)    
plt.show()  